# Exploración preliminar y transformación de datos

In [2]:
# Bibliotecas requeridas
import sys
sys.path.append('../scr/')

import loadAndTransformData as ltd
import viewDataFunctions as vdf


import pandas as pd
import os


In [3]:
# Carga de datos
data = ltd.cargar_csv('C:/Users/jpmog/Downloads/tu_(1)_2024_08_31.csv')

Archivo cargado correctamente.


In [4]:
data.shape

(152781, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152781 entries, 0 to 152780
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   fecha                 152781 non-null  object 
 1   codigo_factura        152781 non-null  int64  
 2   mnt_total_a_pagar     152781 non-null  float64
 3   cant_items            152781 non-null  int64  
 4   familia               152524 non-null  object 
 5   categoria             152781 non-null  object 
 6   subcategoria          152781 non-null  object 
 7   codigo_producto       152781 non-null  float64
 8   cantidad_vendida      152781 non-null  float64
 9   venta_bruta_producto  152781 non-null  float64
 10  venta_neta_producto   152781 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 12.8+ MB


In [6]:
ltd.dividir_y_guardar_csv(data, 'fecha', 'mes', '../data/raw/', 'Minimarket_Sales_Data')

Archivo guardado: ../data/raw/Minimarket_Sales_Data_2024_01.csv
Archivo guardado: ../data/raw/Minimarket_Sales_Data_2024_02.csv
Archivo guardado: ../data/raw/Minimarket_Sales_Data_2024_03.csv


In [11]:
model_data_producto = ltd.pivotear_df(data, 'codigo_factura', 'codigo_producto')
model_data_familia = ltd.pivotear_df(data, 'codigo_factura', 'familia')
model_data_categoria = ltd.pivotear_df(data, 'codigo_factura', 'categoria')
model_data_subcategoria = ltd.pivotear_df(data, 'codigo_factura', 'subcategoria')

In [12]:
print(model_data_producto.shape, model_data_familia.shape, model_data_categoria.shape, model_data_subcategoria.shape)

(39482, 1089) (39225, 12) (39482, 32) (39482, 186)


In [ ]:
ltd.guardar_csv(model_data_producto, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Product')
ltd.guardar_csv(model_data_familia, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Family')
ltd.guardar_csv(model_data_categoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Category')
ltd.guardar_csv(model_data_subcategoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Subcategory')

In [ ]:
dat